In [33]:
import yfinance as yf
import pandas as pd

class Stock:
    def __init__(self, symbol, start_date, end_date):
        self.symbol = symbol
        self.start_date = start_date
        self.end_date = end_date
        self.historical_prices = self.get_historical_prices()

    def get_historical_prices(self):
        data = yf.download(self.symbol, start=self.start_date, end=self.end_date)['Adj Close']
        return data

    def cur_price(self, cur_date):
        return self.historical_prices.loc[cur_date, 'Adj Close']

    def n_day_ret(self, n, cur_date):
        start_date = pd.to_datetime(cur_date) - pd.DateOffset(days=n)
        end_date = pd.to_datetime(cur_date)
        return (self.historical_prices.loc[end_date, 'Adj Close'] /
                self.historical_prices.loc[start_date, 'Adj Close'] - 1) * 100

    def daily_ret(self, cur_date):
        return self.historical_prices.pct_change().loc[cur_date, 'Adj Close'] * 100

    def last_30_days_price(self, cur_date):
        end_date = pd.to_datetime(cur_date)
        start_date = end_date - pd.DateOffset(days=30)
        return self.historical_prices.loc[start_date:end_date, 'Adj Close'].values

# Example Usage
stock_symbol = 'WIPRO'
start_date = '2022-01-01'
end_date = '2022-12-31'
apple_stock = Stock(stock_symbol, start_date, end_date)


[*********************100%***********************]  1 of 1 completed

1 Failed download:
- WIPRO: No timezone found, symbol may be delisted


In [34]:
class ActiveStockSelection:
    def __init__(self, stocks):
        self.stocks = stocks

    def select_portfolio(self, cur_date):
        selected_stocks = []
        for stock in self.stocks:
            if stock.n_day_ret(30, cur_date) > 0:
                selected_stocks.append(stock.symbol)
        return selected_stocks

# Example Usage
nifty50_symbols = 'RELIANCE'  # Add all Nifty 50 symbols
nifty50_stocks = [Stock(symbol, start_date, end_date) for symbol in nifty50_symbols]
active_selection_strategy = ActiveStockSelection(nifty50_stocks)
selected_portfolio = active_selection_strategy.select_portfolio('2023-01-31')
print("Selected Portfolio:", selected_portfolio)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- I: No timezone found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- N: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


IndexingError: Too many indexers

In [27]:
class PerformanceMetrics:
    def __init__(self, benchmark, portfolio, start_date, end_date):
        self.benchmark = benchmark
        self.portfolio = portfolio
        self.start_date = start_date
        self.end_date = end_date

    def calculate_cagr(self, values):
        return ((values[-1] / values[0]) ** (1 / len(values.index.year.unique())) - 1) * 100

    def calculate_volatility(self, daily_returns):
        return (daily_returns.std() * (252 ** 0.5)).values[0]

    def calculate_sharpe_ratio(self, daily_returns):
        return (daily_returns.mean() / daily_returns.std() * (252 ** 0.5)).values[0]

    def get_performance_metrics(self):
        benchmark_returns = self.benchmark.historical_prices.pct_change().dropna()
        portfolio_returns = pd.DataFrame(index=benchmark_returns.index)

        for stock in self.portfolio:
            portfolio_returns[stock.symbol] = stock.historical_prices.pct_change().dropna()

        cagr_benchmark = self.calculate_cagr(self.benchmark.historical_prices)
        volatility_benchmark = self.calculate_volatility(benchmark_returns)
        sharpe_ratio_benchmark = self.calculate_sharpe_ratio(benchmark_returns)

        cagr_portfolio = self.calculate_cagr(portfolio_returns.sum(axis=1) + 1)
        volatility_portfolio = self.calculate_volatility(portfolio_returns.sum(axis=1))
        sharpe_ratio_portfolio = self.calculate_sharpe_ratio(portfolio_returns.sum(axis=1))

        return {
            'CAGR_Benchmark': cagr_benchmark,
            'Volatility_Benchmark': volatility_benchmark,
            'Sharpe_Ratio_Benchmark': sharpe_ratio_benchmark,
            'CAGR_Portfolio': cagr_portfolio,
            'Volatility_Portfolio': volatility_portfolio,
            'Sharpe_Ratio_Portfolio': sharpe_ratio_portfolio
        }

# Example Usage
benchmark_symbol = 'NIFTY50'
benchmark_stock = Stock(benchmark_symbol, start_date, end_date)
performance_metrics = PerformanceMetrics(benchmark_stock, nifty50_stocks, start_date, end_date)
metrics_result = performance_metrics.get_performance_metrics()
print("Performance Metrics:", metrics_result)


[*********************100%***********************]  1 of 1 completed

1 Failed download:
- NIFTY50: No timezone found, symbol may be delisted


IndexError: index -1 is out of bounds for axis 0 with size 0

In [35]:
import dash
from dash import dcc, html

# Assume you have the performance_metrics object from the previous step

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Stock Selection Strategy Performance"),
    
    dcc.DatePickerRange(
        id='date-picker',
        start_date=start_date,
        end_date=end_date
    ),

    dcc.Graph(id='performance-chart')
])

@app.callback(
    dash.dependencies.Output('performance-chart', 'figure'),
    [dash.dependencies.Input('date-picker', 'start_date'),
     dash.dependencies.Input('date-picker', 'end_date')]
)
def update_performance_chart(start_date, end_date):
    performance_metrics = PerformanceMetrics(benchmark_stock, nifty50_stocks, start_date, end_date)
    metrics_result = performance_metrics.get_performance_metrics()

    # Create and return the figure for the performance chart

if __name__ == '__main__':
    app.run_server(debug=True)


---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
Cell In[27], line 24, in PerformanceMetrics.get_performance_metrics(
    self=<__main__.PerformanceMetrics object>
)
     21 for stock in self.portfolio:
     22     portfolio_returns[stock.symbol] = stock.historical_prices.pct_change().dropna()
---> 24 cagr_benchmark = self.calculate_cagr(self.benchmark.historical_prices)
        self = <__main__.PerformanceMetrics object at 0x000001C9926ACED0>
        self.benchmark.historical_prices = Series([], Name: Adj Close, dtype: float64)
        self.benchmark = <__main__.Stock object at 0x000001C98F771290>
     25 volatility_benchmark = self.calculate_volatility(benchmark_returns)
     26 sharpe_ratio_benchmark = self.calculate_sharpe_ratio(benchmark_returns)

Cell In[27], line 9, in PerformanceMetrics.calculate_cagr(
    self=<__main__.PerformanceMetrics object>,
    values=Series([], Name: 